In [1]:
validate_or_submit = 'submit' # 'validate' or 'submit' or 'stresstest'
verbose = True

In [2]:
# =============================================================================
# CELL: Imports (Replace entire imports cell)
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange, tqdm
import itertools
import warnings
import json
import os
import lightgbm
import ast
import gc  # NEW: For memory cleanup
from collections import defaultdict  # NEW: For adaptive thresholding
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin, BaseEstimator, clone
from sklearn.model_selection import cross_val_predict, GroupKFold, train_test_split, StratifiedShuffleSplit  # NEW
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Suppress NumPy warnings for cleaner output (NaN/Inf in feature engineering is expected and handled)
warnings.filterwarnings('ignore', category=RuntimeWarning, module='pandas.core.arraylike')
warnings.filterwarnings('ignore', category=RuntimeWarning, module='pandas.core.computation')
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost.core')

# Try importing CatBoost (optional)
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except:
    CATBOOST_AVAILABLE = False
    print("CatBoost not available")

# Check GPU availability for both PyTorch and LightGBM
GPU_AVAILABLE = False
LGBM_GPU_AVAILABLE = False

# Check PyTorch CUDA
try:
    import torch
    if torch.cuda.is_available():
        GPU_AVAILABLE = True
        print(f"✓ PyTorch GPU Available: {torch.cuda.get_device_name(0)}")
except:
    print("✗ PyTorch GPU not available")

# Check LightGBM GPU support
try:
    import lightgbm as lgb
    # Test if GPU device is available for LightGBM
    test_data = lgb.Dataset(np.random.rand(100, 10), label=np.random.randint(0, 2, 100))
    test_params = {'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0, 'verbose': -1}
    lgb.train(test_params, test_data, num_boost_round=1)
    LGBM_GPU_AVAILABLE = True
    print("✓ LightGBM GPU support detected")
except Exception as e:
    print(f"✗ LightGBM GPU not available: {str(e)[:80]}")
    LGBM_GPU_AVAILABLE = False

print(f"\n{'='*50}")
print(f"GPU Configuration:")
print(f"  - XGBoost will use: {'GPU (cuda:0)' if GPU_AVAILABLE else 'CPU (hist)'}")
print(f"  - LightGBM will use: {'GPU' if LGBM_GPU_AVAILABLE else 'CPU'}")
print(f"  - CatBoost will use: {'GPU' if GPU_AVAILABLE and CATBOOST_AVAILABLE else 'CPU'}")
print(f"{'='*50}\n")

SEED = 42
np.random.seed(SEED)

✓ PyTorch GPU Available: Tesla T4


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


✓ LightGBM GPU support detected

GPU Configuration:
  - XGBoost will use: GPU (cuda:0)
  - LightGBM will use: GPU
  - CatBoost will use: GPU



In [3]:
# =============================================================================
# CELL: StratifiedSubsetClassifier (Replace TrainOnSubsetClassifier cell)
# =============================================================================

class StratifiedSubsetClassifier(ClassifierMixin, BaseEstimator):
    """Enhanced classifier with stratified sampling for balanced class distribution
    
    Key improvements over TrainOnSubsetClassifier:
    - Uses StratifiedShuffleSplit to maintain class balance
    - Better edge case handling (single class, GPU fallback)
    - Memory efficient with float32
    """
    def __init__(self, estimator, n_samples=50_000):
        self.estimator = estimator
        self.n_samples = n_samples

    def _to_numpy(self, X):
        """Convert to numpy float32 for memory efficiency"""
        try:
            return X.to_numpy(np.float32, copy=False)
        except AttributeError:
            return np.asarray(X, dtype=np.float32)

    def fit(self, X, y):
        Xn = self._to_numpy(X)
        y = np.asarray(y).ravel()

        # If n_samples is None → fit on full data
        if self.n_samples is None or len(Xn) <= int(self.n_samples):
            self.estimator.fit(Xn, y)
        else:
            # Stratified sampling to maintain class balance
            sss = StratifiedShuffleSplit(n_splits=1, train_size=int(self.n_samples), random_state=SEED)
            try:
                idx, _ = next(sss.split(np.zeros_like(y), y))
                self.estimator.fit(Xn[idx], y[idx])
            except Exception as e:
                # Fallback to simple subsampling if stratification fails
                print(f"  Stratification failed, using step sampling: {str(e)[:50]}")
                step = max(len(Xn) // int(self.n_samples), 1)
                self.estimator.fit(Xn[::step], y[::step])

        try:
            self.classes_ = np.asarray(self.estimator.classes_)
        except Exception:
            self.classes_ = np.unique(y)
        return self

    def predict_proba(self, X):
        Xn = self._to_numpy(X)
        try:
            P = self.estimator.predict_proba(Xn)
        except Exception:
            # Handle edge cases
            if len(self.classes_) == 1:
                n = len(Xn)
                c = int(self.classes_[0])
                if c == 1:
                    return np.column_stack([np.zeros(n, dtype=np.float32), np.ones(n, dtype=np.float32)])
                else:
                    return np.column_stack([np.ones(n, dtype=np.float32), np.zeros(n, dtype=np.float32)])
            return np.full((len(Xn), 2), 0.5, dtype=np.float32)

        P = np.asarray(P, dtype=np.float32)
        if P.ndim == 1:
            P1 = P.astype(np.float32)
            return np.column_stack([1.0 - P1, P1])
        if P.shape[1] == 1 and len(self.classes_) == 2:
            P1 = P[:, 0].astype(np.float32)
            return np.column_stack([1.0 - P1, P1])
        return P

    def predict(self, X):
        Xn = self._to_numpy(X)
        try:
            return self.estimator.predict(Xn)
        except Exception:
            return np.argmax(self.predict_proba(Xn), axis=1)

In [4]:
"""F Beta customized for the data format of the MABe challenge."""

import json

from collections import defaultdict

import pandas as pd
import polars as pl


class HostVisibleError(Exception):
    pass


def single_lab_f1(lab_solution: pl.DataFrame, lab_submission: pl.DataFrame, beta: float = 1) -> float:
    label_frames: defaultdict[str, set[int]] = defaultdict(set) # key is video/agent/target/action from solution
    prediction_frames: defaultdict[str, set[int]] = defaultdict(set) # key is video/agent/target/action from submission

    for row in lab_solution.to_dicts():
        label_frames[row['label_key']].update(range(row['start_frame'], row['stop_frame']))

    for video in lab_solution['video_id'].unique():
        active_labels: str = lab_solution.filter(pl.col('video_id') == video)['behaviors_labeled'].first()  # ty: ignore
        active_labels: set[str] = set(json.loads(active_labels)) # set of agent,target,action from solution
        predicted_mouse_pairs: defaultdict[str, set[int]] = defaultdict(set) # key is agent,target from submission

        for row in lab_submission.filter(pl.col('video_id') == video).to_dicts(): # every submission row is converted to a dict
            # Since the labels are sparse, we can't evaluate prediction keys not in the active labels.
            if ','.join([str(row['agent_id']), str(row['target_id']), row['action']]) not in active_labels:
                # print(f'ignoring {video}', ','.join([str(row['agent_id']), str(row['target_id']), row['action']]), active_labels)
                continue # these submission rows are ignored
           
            new_frames = set(range(row['start_frame'], row['stop_frame']))
            # Ignore truly redundant predictions.
            new_frames = new_frames.difference(prediction_frames[row['prediction_key']])
            prediction_pair = ','.join([str(row['agent_id']), str(row['target_id'])])
            if predicted_mouse_pairs[prediction_pair].intersection(new_frames):
                # A single agent can have multiple targets per frame (ex: evading all other mice) but only one action per target per frame.
                raise HostVisibleError('Multiple predictions for the same frame from one agent/target pair')
            prediction_frames[row['prediction_key']].update(new_frames)
            predicted_mouse_pairs[prediction_pair].update(new_frames)

    tps = defaultdict(int) # key is action
    fns = defaultdict(int) # key is action
    fps = defaultdict(int) # key is action
    for key, pred_frames in prediction_frames.items():
        action = key.split('_')[-1]
        matched_label_frames = label_frames[key]
        tps[action] += len(pred_frames.intersection(matched_label_frames))
        fns[action] += len(matched_label_frames.difference(pred_frames))
        fps[action] += len(pred_frames.difference(matched_label_frames))

    distinct_actions = set()
    for key, frames in label_frames.items():
        action = key.split('_')[-1]
        distinct_actions.add(action)
        if key not in prediction_frames:
            fns[action] += len(frames)

    action_f1s = []
    for action in distinct_actions:
        # print(f"{tps[action]:8} {fns[action]:8} {fps[action]:8}")
        if tps[action] + fns[action] + fps[action] == 0:
            action_f1s.append(0)
        else:
            action_f1s.append((1 + beta**2) * tps[action] / ((1 + beta**2) * tps[action] + beta**2 * fns[action] + fps[action]))
    return sum(action_f1s) / len(action_f1s)


def mouse_fbeta(solution: pd.DataFrame, submission: pd.DataFrame, beta: float = 1) -> float:
    """
    Doctests:
    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10},
    ... ])
    >>> mouse_fbeta(solution, submission)
    1.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 0, 'stop_frame': 10}, # Wrong action
    ... ])
    >>> mouse_fbeta(solution, submission)
    0.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9},
    ... ])
    >>> "%.12f" % mouse_fbeta(solution, submission)
    '0.500000000000'

    >>> solution = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 345, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9, 'lab_id': 2, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 345, 'agent_id': 1, 'target_id': 2, 'action': 'mount', 'start_frame': 15, 'stop_frame': 24, 'lab_id': 2, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 123, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 9},
    ... ])
    >>> "%.12f" % mouse_fbeta(solution, submission)
    '0.250000000000'

    >>> # Overlapping solution events, one prediction matching both.
    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 10, 'stop_frame': 20, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 20},
    ... ])
    >>> mouse_fbeta(solution, submission)
    1.0

    >>> solution = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 10, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 30, 'stop_frame': 40, 'lab_id': 1, 'behaviors_labeled': '["1,2,attack"]'},
    ... ])
    >>> submission = pd.DataFrame([
    ...     {'video_id': 1, 'agent_id': 1, 'target_id': 2, 'action': 'attack', 'start_frame': 0, 'stop_frame': 40},
    ... ])
    >>> mouse_fbeta(solution, submission)
    0.6666666666666666
    """
    if len(solution) == 0 or len(submission) == 0:
        raise ValueError('Missing solution or submission data')

    expected_cols = ['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']

    for col in expected_cols:
        if col not in solution.columns:
            raise ValueError(f'Solution is missing column {col}')
        if col not in submission.columns:
            raise ValueError(f'Submission is missing column {col}')

    solution: pl.DataFrame = pl.DataFrame(solution)
    submission: pl.DataFrame = pl.DataFrame(submission)
    assert (solution['start_frame'] <= solution['stop_frame']).all()
    assert (submission['start_frame'] <= submission['stop_frame']).all()
    solution_videos = set(solution['video_id'].unique())
    # Need to align based on video IDs as we can't rely on the row IDs for handling public/private splits.
    submission = submission.filter(pl.col('video_id').is_in(solution_videos))

    solution = solution.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('label_key'),
    )
    submission = submission.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('prediction_key'),
    )

    lab_scores = []
    for lab in solution['lab_id'].unique():
        lab_solution = solution.filter(pl.col('lab_id') == lab).clone()
        lab_videos = set(lab_solution['video_id'].unique())
        lab_submission = submission.filter(pl.col('video_id').is_in(lab_videos)).clone()
        lab_scores.append(single_lab_f1(lab_solution, lab_submission, beta=beta))

    return sum(lab_scores) / len(lab_scores)


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, beta: float = 1) -> float:
    """
    F1 score for the MABe Challenge
    """
    solution = solution.drop(row_id_column_name, axis='columns', errors='ignore')
    submission = submission.drop(row_id_column_name, axis='columns', errors='ignore')
    return mouse_fbeta(solution, submission, beta=beta)

In [5]:
train = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/train.csv')
train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)
train_without_mabe22 = train.query("~ lab_id.str.startswith('MABe22_')")

test = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/test.csv')

# labs = list(np.unique(train.lab_id))

body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

# behaviors = list(train.behaviors_labeled.drop_duplicates().dropna())
# behaviors = sorted(list({b.replace("'", "") for bb in behaviors for b in json.loads(bb)}))
# behaviors = [b.split(',') for b in behaviors]
# behaviors = pd.DataFrame(behaviors, columns=['agent', 'target', 'action'])


In [6]:
def create_solution_df(dataset):
    """Create the solution dataframe for validating out-of-fold predictions.

    From https://www.kaggle.com/code/ambrosm/mabe-validated-baseline-without-machine-learning/
    
    Parameters:
    dataset: (a subset of) the train dataframe
    
    Return values:
    solution: solution dataframe in the correct format for the score() function
    """
    solution = []
    for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
    
        # Load annotation file
        lab_id = row['lab_id']
        if lab_id.startswith('MABe22'): continue
        video_id = row['video_id']
        path = f"/kaggle/input/MABe-mouse-behavior-detection/train_annotation/{lab_id}/{video_id}.parquet"
        try:
            annot = pd.read_parquet(path)
        except FileNotFoundError:
            # MABe22 and one more training file lack annotations.
            if verbose: print(f"No annotations for {path}")
            continue
    
        # Add all annotations to the solution
        annot['lab_id'] = lab_id
        annot['video_id'] = video_id
        annot['behaviors_labeled'] = row['behaviors_labeled']
        annot['target_id'] = np.where(annot.target_id != annot.agent_id, annot['target_id'].apply(lambda s: f"mouse{s}"), 'self')
        annot['agent_id'] = annot['agent_id'].apply(lambda s: f"mouse{s}")
        solution.append(annot)
    
    solution = pd.concat(solution)
    return solution

if validate_or_submit == 'validate':
    solution = create_solution_df(train_without_mabe22)

In [7]:
if validate_or_submit == 'stresstest':
    n_videos_per_lab = 2
    
    try:
        os.mkdir(f"stresstest_tracking")
    except FileExistsError:
        pass
    
    rng = np.random.default_rng()
    stresstest = pd.concat(
        [train.query("video_id == 1459695188")] # long video from BoisterousParrot
        + [df.sample(min(n_videos_per_lab, len(df)), random_state=1) for (_, df) in train.groupby('lab_id')])
    for _, row in tqdm(stresstest.iterrows(), total=len(stresstest)):
        lab_id = row['lab_id']
        video_id = row['video_id']
        
        # Load video
        path = f"/kaggle/input/MABe-mouse-behavior-detection/train_tracking/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
    
        if video_id == 1459695188: # long video from BoisterousParrot
            vid = pd.concat([vid] * 3) # provoke out of memory (5 is too much)
            vid['video_frame'] = np.arange(len(vid))
    
        # Drop some complete frames
        dropped_frames = list(rng.choice(np.unique(vid.video_frame), size=100, replace=False))
        vid = vid.query("~ video_frame.isin(@dropped_frames)")
        
        # Drop a complete bodypart
        if rng.uniform() < 0.2:
            dropped_bodypart = rng.choice(np.unique(vid.bodypart), size=1, replace=False)[0]
            vid = vid.query("bodypart != @dropped_bodypart")
        
        # Drop a mouse
        if rng.uniform() < 0.1:
            vid = vid.query("mouse_id != 1")
        
        # Drop random bodyparts from random frames
        if rng.uniform() < 0.7:
            mask = np.ones(len(vid), dtype=bool)
            mask[:int(0.4 * len(mask))] = False
            rng.shuffle(mask)
            vid = vid[mask]
    
        # Set random coordinates of bodyparts to nan
        if rng.uniform() < 0.7:
            mask = np.ones(len(vid), dtype=bool)
            mask[:int(0.2 * len(mask))] = False
            rng.shuffle(mask)
            vid.loc[:, 'x'] = np.where(mask, np.nan, vid.loc[:, 'x'])
            rng.shuffle(mask)
            vid.loc[:, 'y'] = np.where(mask, np.nan, vid.loc[:, 'y'])
    
        # Save the video
        try:
            os.mkdir(f"stresstest_tracking/{lab_id}")
        except FileExistsError:
            pass
        new_path = f"stresstest_tracking/{lab_id}/{video_id}.parquet"
        vid.to_parquet(new_path)


In [8]:
drop_body_parts =  ['headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
                    'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
                    'spine_1', 'spine_2',
                    'tail_middle_1', 'tail_middle_2', 'tail_midpoint']

def generate_mouse_data(dataset, traintest, traintest_directory=None, generate_single=True, generate_pair=True):
    """Generate batches of data in coordinate representation.

    The batches have variable length, and every batch can have other columns
    for the labels, depending on what behaviors
    were labeled for the batch.

    Every video can produce zero, one or two batches.
    
    Parameters
    ----------
    dataset: (subset of) train.csv or test.csv dataframe
    traintest: either 'train' or 'test'

    Yields
    ------
    switch: either 'single' or 'pair'
    data: dataframe containing coordinates of the body parts of a single mouse or of a pair of mice
    meta: dataframe with columns ['video_id', 'agent_id', 'target_id', 'video_frame', 'fps']
    label: dataframe with labels (0, 1), one column per action, only if traintest == 'train'
    actions: list of actions to be predicted for this batch, only if traintest == 'test'
    """
    assert traintest in ['train', 'test']
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"
    for _, row in dataset.iterrows():
        
        # Load the video and pivot it sn that one frame = one row
        lab_id = row.lab_id
        if lab_id.startswith('MABe22'): continue
        video_id = row.video_id

        if type(row.behaviors_labeled) != str:
            # We cannot use videos without labeled behaviors
            print('No labeled behaviors:', lab_id, video_id, type(row.behaviors_labeled), row.behaviors_labeled)
            continue

        # Get FPS for this video
        fps = row.get('fps_approx', 30.0)  # Default to 30 if not available

        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~ bodypart.isin(@drop_body_parts)")
        pvid = vid.pivot(columns=['mouse_id', 'bodypart'], index='video_frame', values=['x', 'y'])
        if pvid.isna().any().any():
            if verbose and traintest == 'test': print('video with missing values', video_id, traintest, len(vid), 'frames')
        else:
            if verbose and traintest == 'test': print('video with all values', video_id, traintest, len(vid), 'frames')
        del vid
        pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T # mouse_id, body_part, xy
        pvid /= row.pix_per_cm_approx # convert to cm

        # Determine the behaviors of this video
        vid_behaviors = json.loads(row.behaviors_labeled)
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
        
        # Load the annotations
        if traintest == 'train':
            try:
                annot = pd.read_parquet(path.replace('train_tracking', 'train_annotation'))
            except FileNotFoundError:
                # MABe22 and one more training file lack annotations.
                # We simply drop these videos.
                continue

        # Create the single_mouse dataframes: single_mouse, single_mouse_label and single_mouse_meta
        if generate_single:
            vid_behaviors_subset = vid_behaviors.query("target == 'self'") # single-mouse behaviors of this video
            for mouse_id_str in np.unique(vid_behaviors_subset.agent):
                try:
                    mouse_id = int(mouse_id_str[-1])
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("agent == @mouse_id_str").action)
                    single_mouse = pvid.loc[:, mouse_id]
                    assert len(single_mouse) == len(pvid)
                    single_mouse_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': mouse_id_str,
                        'target_id': 'self',
                        'video_frame': single_mouse.index,
                        'fps': fps
                    })
                    if traintest == 'train':
                        single_mouse_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=single_mouse.index)
                        annot_subset = annot.query("(agent_id == @mouse_id) & (target_id == @mouse_id)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            single_mouse_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'single', single_mouse, single_mouse_meta, single_mouse_label
                    else:
                        if verbose: print('- test single', video_id, mouse_id)
                        yield 'single', single_mouse, single_mouse_meta, vid_agent_actions
                except KeyError:
                    pass # If there is no data for the selected agent mouse, we skip the mouse.

        # Create the mouse_pair dataframes: mouse_pair, mouse_label and mouse_meta
        if generate_pair:
            vid_behaviors_subset = vid_behaviors.query("target != 'self'")
            if len(vid_behaviors_subset) > 0:
                for agent, target in itertools.permutations(np.unique(pvid.columns.get_level_values('mouse_id')), 2): # int8
                    agent_str = f"mouse{agent}"
                    target_str = f"mouse{target}"
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("(agent == @agent_str) & (target == @target_str)").action)
                    mouse_pair = pd.concat([pvid[agent], pvid[target]], axis=1, keys=['A', 'B'])
                    assert len(mouse_pair) == len(pvid)
                    mouse_pair_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': agent_str,
                        'target_id': target_str,
                        'video_frame': mouse_pair.index,
                        'fps': fps
                    })
                    if traintest == 'train':
                        mouse_pair_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=mouse_pair.index)
                        annot_subset = annot.query("(agent_id == @agent) & (target_id == @target)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            mouse_pair_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'pair', mouse_pair, mouse_pair_meta, mouse_pair_label
                    else:
                        if verbose: print('- test pair', video_id, agent, target)
                        yield 'pair', mouse_pair, mouse_pair_meta, vid_agent_actions


In [9]:
# =============================================================================
# CELL: Adaptive Thresholding (Add before predict_multiclass)
# =============================================================================

# ==================== ADAPTIVE THRESHOLDING ====================

action_thresholds = {
    "default": 0.25,  # Lower threshold for better recall
    "single_default": 0.25,
    "pair_default": 0.25,
    "single": {
        "rear": 0.27,
        "rest": 0.22,
        "selfgroom": 0.23,
    },
    "pair": {
        "attack": 0.23,
        "mount": 0.26,
        "chase": 0.24,
        "sniff": 0.22,
    }
}

def _select_threshold_map(thresholds, mode: str):
    """Select threshold based on mode (single/pair) and action"""
    if isinstance(thresholds, dict):
        # Check if mode-specific thresholds exist
        if ("single" in thresholds) or ("pair" in thresholds):
            base_default = float(thresholds.get("default", 0.25))
            mode_default = float(thresholds.get(f"{mode}_default", base_default))
            mode_overrides = thresholds.get(mode, {}) or {}
            out = defaultdict(lambda: mode_default)
            out.update({str(k): float(v) for k, v in mode_overrides.items()})
            return out
        # Plain per-action dict
        out = defaultdict(lambda: float(thresholds.get("default", 0.25)))
        out.update({str(k): float(v) for k, v in thresholds.items() if k != "default"})
        return out
    return defaultdict(lambda: 0.25)

# Helper functions for FPS-aware feature engineering
def _scale(window, fps, base_fps=30.0):
    """Scale window size by FPS ratio"""
    return max(1, int(round(window * fps / base_fps)))

def _scale_signed(offset, fps, base_fps=30.0):
    """Scale signed offset by FPS ratio"""
    sign = 1 if offset >= 0 else -1
    return sign * max(1, int(round(abs(offset) * fps / base_fps)))


In [10]:
# =============================================================================
# CELL: Enhanced predict_multiclass (Replace predict_multiclass cell)
# =============================================================================

# Make the multi-class prediction with adaptive thresholding
def predict_multiclass_adaptive(pred, meta, action_thresholds):
    """Enhanced multiclass prediction with temporal smoothing and adaptive thresholding
    
    Improvements:
    - Temporal smoothing (rolling window) to reduce noise
    - Action-specific thresholds
    - Filter short events (< 2 frames) - more sensitive
    
    Parameters
    ----------
    pred: dataframe of predicted binary probabilities, shape (n_samples, n_actions)
    meta: dataframe with columns ['video_id', 'agent_id', 'target_id', 'video_frame', 'fps']
    action_thresholds: dict with thresholds per action
    """
    # Get FPS for adaptive smoothing
    fps = meta['fps'].iloc[0] if 'fps' in meta.columns else 30.0
    
    # Apply FPS-aware temporal smoothing to reduce noise
    smooth_window = _scale(5, fps)
    pred_smoothed = pred.rolling(window=smooth_window, min_periods=1, center=True).mean()
    
    # Determine mode (single vs pair)
    mode = 'pair'
    try:
        if 'target_id' in meta.columns and meta['target_id'].eq('self').all():
            mode = 'single'
    except Exception:
        pass
    
    # Get adaptive threshold map for this mode
    th_map = _select_threshold_map(action_thresholds, mode)
    
    # Find most probable action
    ama = np.argmax(pred_smoothed, axis=1)
    max_probs = pred_smoothed.max(axis=1)
    
    # Apply action-specific thresholds
    threshold_mask = np.zeros(len(pred_smoothed), dtype=bool)
    for i, action in enumerate(pred_smoothed.columns):
        action_mask = (ama == i)
        threshold = th_map[action]
        threshold_mask |= (action_mask & (max_probs >= threshold))
    
    ama = np.where(threshold_mask, ama, -1)
    ama = pd.Series(ama, index=meta.video_frame)
    
    # Keep only start and stop frames
    changes_mask = (ama != ama.shift(1)).values
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]
    mask = ama_changes.values >= 0
    mask[-1] = False
    
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': pred.columns[ama_changes[mask].values],
        'start_frame': ama_changes.index[mask],
        'stop_frame': ama_changes.index[1:][mask[:-1]]
    })
    
    stop_video_id = meta_changes['video_id'][1:][mask[:-1]].values
    stop_agent_id = meta_changes['agent_id'][1:][mask[:-1]].values
    stop_target_id = meta_changes['target_id'][1:][mask[:-1]].values
    
    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]
        if i < len(stop_video_id):
            if stop_video_id[i] != video_id or stop_agent_id[i] != agent_id or stop_target_id[i] != target_id:
                new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
                submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
        else:
            new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
            submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
    
    # Filter out very short events (reduced from 3 to 2 for better recall)
    if len(submission_part) > 0:
        duration = submission_part.stop_frame - submission_part.start_frame
        submission_part = submission_part[duration >= 2].reset_index(drop=True)
        if len(submission_part) > 0:
            assert (submission_part.stop_frame > submission_part.start_frame).all(), 'stop <= start'
    
    if verbose: print(f'  actions found: {len(submission_part)}')
    return submission_part

# Legacy function for backward compatibility
def predict_multiclass(pred, meta):
    """Legacy wrapper - uses adaptive thresholding"""
    return predict_multiclass_adaptive(pred, meta, action_thresholds)


In [11]:
# =============================================================================
# CELL: FPS-Aware Advanced Feature Engineering
# =============================================================================

def add_curvature_features(X, center_x, center_y, fps):
    """Add curvature and turning rate features"""
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    
    # Angle and turning
    angle = np.arctan2(vel_y, vel_x)
    angle_change = np.abs(angle.diff())
    
    for w in [5, 10, 20]:
        ws = _scale(w, fps)
        X[f'turn_rate_{w}'] = angle_change.rolling(ws, min_periods=max(1, ws // 5)).sum()
    
    return X

def add_multiscale_features(X, center_x, center_y, fps):
    """Add multi-scale temporal features"""
    for w in [3, 7, 15]:
        ws = _scale(w, fps)
        roll_kwargs = dict(min_periods=1, center=True)
        
        # Position statistics at different scales
        X[f'x_std_{w}'] = center_x.rolling(ws, **roll_kwargs).std()
        X[f'y_std_{w}'] = center_y.rolling(ws, **roll_kwargs).std()
        X[f'x_skew_{w}'] = center_x.rolling(ws, **roll_kwargs).apply(lambda x: pd.Series(x).skew(), raw=False)
    
    return X

def add_state_features(X, center_x, center_y, fps):
    """Add behavioral state features (static, slow, fast)"""
    # Velocity magnitude
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    speed = np.sqrt(vel_x**2 + vel_y**2)
    
    w = _scale(15, fps)
    speed_smooth = speed.rolling(w, min_periods=1, center=True).mean()
    
    # State classification
    X['is_static'] = (speed_smooth < 0.5).astype(float)
    X['is_slow'] = ((speed_smooth >= 0.5) & (speed_smooth < 2.0)).astype(float)
    X['is_fast'] = (speed_smooth >= 2.0).astype(float)
    
    return X

def add_longrange_features(X, center_x, center_y, fps):
    """Add long-range temporal features"""
    for lag in [60, 120]:
        l = _scale(lag, fps)
        X[f'x_lag_{lag}'] = center_x.shift(l)
        X[f'y_lag_{lag}'] = center_y.shift(l)
        X[f'dist_from_{lag}'] = np.sqrt((center_x - center_x.shift(l))**2 + 
                                        (center_y - center_y.shift(l))**2)
    
    return X

def add_interaction_features(X, mouse_pair, avail_A, avail_B, fps):
    """Add advanced interaction features for pairs"""
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X
    
    # Distance dynamics
    dist = np.sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                   (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)
    
    for w in [10, 30]:
        ws = _scale(w, fps)
        X[f'dist_accel_{w}'] = dist.diff().diff().rolling(ws, min_periods=1).mean()
    
    # Relative position change
    if 'nose' in avail_A and 'nose' in avail_B:
        rel_vel_x = (mouse_pair['A']['nose']['x'].diff() - mouse_pair['B']['nose']['x'].diff())
        rel_vel_y = (mouse_pair['A']['nose']['y'].diff() - mouse_pair['B']['nose']['y'].diff())
        X['rel_speed'] = np.sqrt(rel_vel_x**2 + rel_vel_y**2)
    
    return X

def transform_single(single_mouse, body_parts_tracked, fps=30.0):
    """Transform from cartesian coordinates to distance representation with FPS awareness.

    Parameters:
    single_mouse: dataframe with coordinates of the body parts of one mouse
                  shape (n_samples, n_body_parts * 2)
                  two-level MultiIndex on columns
    body_parts_tracked: list of body parts
    fps: frames per second of the video (for temporal feature scaling)
    """
    available_body_parts = single_mouse.columns.get_level_values(0)
    X = pd.DataFrame({
            f"{part1}+{part2}": np.square(single_mouse[part1] - single_mouse[part2]).sum(axis=1, skipna=False)
            for part1, part2 in itertools.combinations(body_parts_tracked, 2) if part1 in available_body_parts and part2 in available_body_parts
        })
    X = X.reindex(columns=[f"{part1}+{part2}" for part1, part2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    # Speed features with FPS-aware lag
    if 'ear_left' in single_mouse.columns and 'ear_right' in single_mouse.columns and 'tail_base' in single_mouse.columns:
        lag = _scale(10, fps)
        shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(lag)
        X = pd.concat([
            X, 
            pd.DataFrame({
                'speed_left': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
                'speed_right': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
                'speed_left2': np.square(single_mouse['ear_left'] - shifted['tail_base']).sum(axis=1, skipna=False),
                'speed_right2': np.square(single_mouse['ear_right'] - shifted['tail_base']).sum(axis=1, skipna=False),
            })
        ], axis=1)

    # Body angle (orientation)
    if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose'] - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        X['body_ang'] = (v1['x'] * v2['x'] + v1['y'] * v2['y']) / (
            np.sqrt(v1['x']**2 + v1['y']**2) * np.sqrt(v2['x']**2 + v2['y']**2) + 1e-6)

    # Core temporal features with FPS scaling
    if 'body_center' in available_body_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']

        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'cx_m{w}'] = cx.rolling(ws, **roll).mean()
            X[f'cy_m{w}'] = cy.rolling(ws, **roll).mean()
            X[f'cx_s{w}'] = cx.rolling(ws, **roll).std()
            X[f'cy_s{w}'] = cy.rolling(ws, **roll).std()
            X[f'x_rng{w}'] = cx.rolling(ws, **roll).max() - cx.rolling(ws, **roll).min()
            X[f'y_rng{w}'] = cy.rolling(ws, **roll).max() - cy.rolling(ws, **roll).min()
            X[f'disp{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).sum()**2 +
                                     cy.diff().rolling(ws, min_periods=1).sum()**2)
            X[f'act{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).var() +
                                   cy.diff().rolling(ws, min_periods=1).var())
            # Additional: median
            X[f'cx_med{w}'] = cx.rolling(ws, **roll).median()
            X[f'cy_med{w}'] = cy.rolling(ws, **roll).median()

        # Advanced features
        X = add_curvature_features(X, cx, cy, fps)
        X = add_multiscale_features(X, cx, cy, fps)
        X = add_state_features(X, cx, cy, fps)
        X = add_longrange_features(X, cx, cy, fps)

    # Nose-tail features with FPS-aware lags
    if all(p in available_body_parts for p in ['nose', 'tail_base']):
        nt_dist = np.sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                          (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nt_lg{lag}'] = nt_dist.shift(l)
            X[f'nt_df{lag}'] = nt_dist - nt_dist.shift(l)

    # Ear features with FPS-aware offsets
    if all(p in available_body_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
                        (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)
        for off in [-30, -20, -10, 10, 20, 30]:
            o = _scale_signed(off, fps)
            X[f'ear_o{off}'] = ear_d.shift(-o)
        w = _scale(30, fps)
        X['ear_con'] = ear_d.rolling(w, min_periods=1, center=True).std() / \
                       (ear_d.rolling(w, min_periods=1, center=True).mean() + 1e-6)

    return X.astype(np.float32, copy=False)

def transform_pair(mouse_pair, body_parts_tracked, fps=30.0):
    """Transform from cartesian coordinates to distance representation with FPS awareness.

    Parameters:
    mouse_pair: dataframe with coordinates of the body parts of two mice
                  shape (n_samples, 2 * n_body_parts * 2)
                  three-level MultiIndex on columns
    body_parts_tracked: list of body parts
    fps: frames per second of the video
    """
    avail_A = mouse_pair['A'].columns.get_level_values(0)
    avail_B = mouse_pair['B'].columns.get_level_values(0)

    # Inter-mouse distances
    X = pd.DataFrame({
        f"12+{p1}+{p2}": np.square(mouse_pair['A'][p1] - mouse_pair['B'][p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.product(body_parts_tracked, repeat=2)
        if p1 in avail_A and p2 in avail_B
    })
    X = X.reindex(columns=[f"12+{p1}+{p2}" for p1, p2 in itertools.product(body_parts_tracked, repeat=2)], copy=False)

    # Speed features with FPS-aware lag
    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        lag = _scale(10, fps)
        shA = mouse_pair['A']['ear_left'].shift(lag)
        shB = mouse_pair['B']['ear_left'].shift(lag)
        speeds = pd.DataFrame({
            'sp_A': np.square(mouse_pair['A']['ear_left'] - shA).sum(axis=1, skipna=False),
            'sp_AB': np.square(mouse_pair['A']['ear_left'] - shB).sum(axis=1, skipna=False),
            'sp_B': np.square(mouse_pair['B']['ear_left'] - shB).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    # Relative orientation
    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        X['rel_ori'] = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']) / (
            np.sqrt(dir_A['x']**2 + dir_A['y']**2) * np.sqrt(dir_B['x']**2 + dir_B['y']**2) + 1e-6)

    # Approach rate with FPS-aware lag
    if all(p in avail_A for p in ['nose']) and all(p in avail_B for p in ['nose']):
        cur = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)
        lag = _scale(10, fps)
        shA_n = mouse_pair['A']['nose'].shift(lag)
        shB_n = mouse_pair['B']['nose'].shift(lag)
        past = np.square(shA_n - shB_n).sum(axis=1, skipna=False)
        X['appr'] = cur - past

    # Distance bins (cm; unchanged by fps)
    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd = np.sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                     (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)
        X['v_cls'] = (cd < 5.0).astype(float)
        X['cls']   = ((cd >= 5.0) & (cd < 15.0)).astype(float)
        X['med']   = ((cd >= 15.0) & (cd < 30.0)).astype(float)
        X['far']   = (cd >= 30.0).astype(float)

    # Temporal interaction features with FPS scaling
    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd_full = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)

        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'd_m{w}']  = cd_full.rolling(ws, **roll).mean()
            X[f'd_s{w}']  = cd_full.rolling(ws, **roll).std()
            X[f'd_mn{w}'] = cd_full.rolling(ws, **roll).min()
            X[f'd_mx{w}'] = cd_full.rolling(ws, **roll).max()
            X[f'd_med{w}'] = cd_full.rolling(ws, **roll).median()

            d_var = cd_full.rolling(ws, **roll).var()
            X[f'int{w}'] = 1 / (1 + d_var)

            Axd = mouse_pair['A']['body_center']['x'].diff()
            Ayd = mouse_pair['A']['body_center']['y'].diff()
            Bxd = mouse_pair['B']['body_center']['x'].diff()
            Byd = mouse_pair['B']['body_center']['y'].diff()
            coord = Axd * Bxd + Ayd * Byd
            X[f'co_m{w}'] = coord.rolling(ws, **roll).mean()
            X[f'co_s{w}'] = coord.rolling(ws, **roll).std()

    # Nose-nose dynamics with FPS-aware lags
    if 'nose' in avail_A and 'nose' in avail_B:
        nn = np.sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
                     (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nn_lg{lag}']  = nn.shift(l)
            X[f'nn_ch{lag}']  = nn - nn.shift(l)
            is_cl = (nn < 10.0).astype(float)
            X[f'cl_ps{lag}']  = is_cl.rolling(l, min_periods=1).mean()

    # Velocity alignment with FPS-aware offsets
    if 'body_center' in avail_A and 'body_center' in avail_B:
        Avx = mouse_pair['A']['body_center']['x'].diff()
        Avy = mouse_pair['A']['body_center']['y'].diff()
        Bvx = mouse_pair['B']['body_center']['x'].diff()
        Bvy = mouse_pair['B']['body_center']['y'].diff()
        val = (Avx * Bvx + Avy * Bvy) / (np.sqrt(Avx**2 + Avy**2) * np.sqrt(Bvx**2 + Bvy**2) + 1e-6)

        for off in [-30, -20, -10, 0, 10, 20, 30]:
            o = _scale_signed(off, fps)
            X[f'va_{off}'] = val.shift(-o)

        w = _scale(30, fps)
        X['int_con'] = cd_full.rolling(w, min_periods=1, center=True).std() / \
                       (cd_full.rolling(w, min_periods=1, center=True).mean() + 1e-6)

        # Advanced interaction
        X = add_interaction_features(X, mouse_pair, avail_A, avail_B, fps)

    return X.astype(np.float32, copy=False)


In [12]:
# =============================================================================
# CELL: Enhanced cross_validate_classifier (Replace in cross_validate_classifier)
# =============================================================================

# ADD this at the beginning of cross_validate_classifier function,
# right after the function definition line:

def cross_validate_classifier(binary_classifier, X, label, meta):
    """Cross-validate a binary classifier per action and a multi-class classifier over all actions.

    Parameters
    ----------
    binary_classifier: classifier with predict_proba (can be a list of classifiers for ensemble)
    X: 2d array-like (distance representation) of shape (n_samples, n_features)
    label: dataframe with binary targets (one column per action, may have missing values), index doesn't matter
    meta: dataframe with columns ['video_id', 'agent_id', 'target_id', 'video_frame'], index doesn't matter

    Output
    ------
    appends to f1_list (binary) and submission_list (multi-class)
    
    """
    # Cross-validate a binary classifier for every action
    oof = pd.DataFrame(index=meta.video_frame) # will get a column per action
    for action in label.columns:
        # Filter for samples (video frames) with a defined target (i.e., target is not nan)
        action_mask = ~ label[action].isna().values
        X_action = X[action_mask]
        y_action = label[action][action_mask].values.astype(int)
        p = y_action.mean()
        baseline_score = p / (1 + p)
        groups_action = meta.video_id[action_mask] # ensure validation has unseen videos
        if len(np.unique(groups_action)) < 5:
            continue # GroupKFold would fail with fewer than n_splits groups

        if not (y_action == 0).all():
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=RuntimeWarning)
                # Use first model from ensemble for cross-validation
                if isinstance(binary_classifier, list):
                    oof_action = cross_val_predict(binary_classifier[0], X_action, y_action, 
                                                   groups=groups_action, cv=GroupKFold(), method='predict_proba')
                else:
                    oof_action = cross_val_predict(binary_classifier, X_action, y_action, 
                                                   groups=groups_action, cv=GroupKFold(), method='predict_proba')
            oof_action = oof_action[:, 1]
        else:
            oof_action = np.zeros(len(y_action))
        
        # Use adaptive threshold for this action
        mode = 'single' if 'target_id' in meta.columns and meta['target_id'].eq('self').all() else 'pair'
        th_map = _select_threshold_map(action_thresholds, mode)
        action_threshold = th_map[action]
        
        f1 = f1_score(y_action, (oof_action >= action_threshold), zero_division=0)
        ch = '>' if f1 > baseline_score else '=' if f1 == baseline_score else '<'
        print(f"  F1: {f1:.3f} {ch} ({baseline_score:.3f}) {action} (th={action_threshold:.3f})")
        f1_list.append((body_parts_tracked_str, action, f1))
        oof_column = np.zeros(len(label))
        oof_column[action_mask] = oof_action
        oof[action] = oof_column

    # Make the multi-class prediction with adaptive thresholding
    submission_part = predict_multiclass_adaptive(oof, meta, action_thresholds)
    submission_list.append(submission_part)

In [13]:
# =============================================================================
# CELL: Enhanced submit function with FPS awareness
# =============================================================================

def submit(body_parts_tracked_str, switch_tr, models, model_weights, X_tr, label, meta):
    """Produce a submission file for the selected subset of the test data.
    
    Enhanced with:
    - Ensemble of multiple models with weighted voting
    - Memory optimization (float32, gc.collect)
    - FPS-aware feature transformation
    - Robust error handling for imbalanced classes
    - LightGBM on CPU for stability, XGBoost/CatBoost on GPU
    
    Parameters
    ----------
    body_parts_tracked_str: subset of body parts for filtering the test set
    switch_tr: 'single' or 'pair'
    models: list of classifiers for ensemble
    model_weights: weights for each model in ensemble
    X_tr: training features as 2d array-like of shape (n_samples, n_features)
    label: dataframe with binary targets (one column per action, may have missing values)
    meta: dataframe with columns ['video_id', 'agent_id', 'target_id', 'video_frame', 'fps']

    Output
    ------
    appends to submission_list
    
    """
    # Train ensemble of models for every action (MEMORY OPTIMIZED)
    X_tr_np = X_tr.to_numpy(np.float32, copy=False) if hasattr(X_tr, 'to_numpy') else np.asarray(X_tr, dtype=np.float32)
    
    model_list = []  # will store (action, [model1, model2, ...]) tuples
    for action in label.columns:
        action_mask = ~ label[action].isna().values
        y_action = label[action][action_mask].values.astype(int)
        
        # Enhanced class balance checking
        n_positives = np.sum(y_action)
        n_total = len(y_action)
        
        # Only train if:
        # - Not all zeros
        # - At least 10 positive samples (increased from 5)
        # - Positive samples < 99% of total (avoid extreme imbalance)
        if not (y_action == 0).all() and n_positives >= 10 and n_positives < n_total * 0.99:
            trained_models = []
            idx = np.flatnonzero(action_mask)
            
            # Train all models in the ensemble with robust error handling
            for i, model in enumerate(models):
                try:
                    model_clone = clone(model)
                    model_clone.fit(X_tr_np[idx], y_action)
                    trained_models.append(model_clone)
                except Exception as e:
                    error_msg = str(e)
                    # Handle specific LightGBM error
                    if 'best_split_info.left_count' in error_msg:
                        if verbose:
                            print(f'  Stratification failed, using step sampling: {error_msg[:50]}...')
                        try:
                            # Fallback to step sampling
                            step = max(len(idx) // 50000, 1)
                            model_clone = clone(model)
                            model_clone.fit(X_tr_np[idx][::step], y_action[::step])
                            trained_models.append(model_clone)
                        except Exception as e2:
                            if verbose:
                                print(f'    Model training failed for {action}: {str(e2)[:50]}...')
                            continue
                    else:
                        if verbose:
                            print(f'    Model training failed for {action}: {error_msg[:50]}...')
                        continue
            
            if len(trained_models) > 0:
                model_list.append((action, trained_models))
            elif verbose:
                print(f'    No models trained successfully for {action}')
    
    # Clean up training data from memory
    del X_tr_np
    gc.collect()

    # Compute test predictions in batches
    body_parts_tracked = json.loads(body_parts_tracked_str)
    if len(body_parts_tracked) > 5:
        body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
    
    if validate_or_submit == 'submit':
        test_subset = test[test.body_parts_tracked == body_parts_tracked_str]
        generator = generate_mouse_data(test_subset, 'test',
                                        generate_single=(switch_tr == 'single'), 
                                        generate_pair=(switch_tr == 'pair'))
    else:
        test_subset = stresstest.query("body_parts_tracked == @body_parts_tracked_str")
        generator = generate_mouse_data(test_subset, 'test',
                                        traintest_directory='stresstest_tracking',
                                        generate_single=(switch_tr == 'single'),
                                        generate_pair=(switch_tr == 'pair'))
    
    if verbose: print(f"n_videos: {len(test_subset)}, n_models: {len(models)}")
    
    for switch_te, data_te, meta_te, actions_te in generator:
        assert switch_te == switch_tr
        try:
            # Get FPS for this video
            fps = meta_te['fps'].iloc[0] if 'fps' in meta_te.columns else 30.0
            
            # Transform from coordinate representation into distance representation (FPS-aware)
            if switch_te == 'single':
                X_te = transform_single(data_te, body_parts_tracked, fps)
            else:
                X_te = transform_pair(data_te, body_parts_tracked, fps)
            
            if verbose and len(X_te) == 0: print("ERROR: X_te is empty")
            
            # MEMORY OPTIMIZATION: Convert to float32 and numpy
            X_te_np = X_te.to_numpy(np.float32, copy=False) if hasattr(X_te, 'to_numpy') else np.asarray(X_te, dtype=np.float32)
            del X_te, data_te
            gc.collect()
            
            # Compute binary predictions using weighted ensemble
            pred = pd.DataFrame(index=meta_te.video_frame)
            for action, trained_models in model_list:
                if action in actions_te:
                    # Get predictions from all models
                    probs = []
                    for model in trained_models:
                        try:
                            probs.append(model.predict_proba(X_te_np)[:, 1])
                        except Exception as e:
                            if verbose: print(f"    Prediction failed: {str(e)[:30]}")
                    
                    if len(probs) > 0:
                        # Weighted average
                        if len(probs) == len(model_weights):
                            pred[action] = np.average(probs, axis=0, weights=model_weights)
                        else:
                            pred[action] = np.mean(probs, axis=0)
            
            del X_te_np
            gc.collect()
            
            # Compute multiclass predictions with adaptive thresholding
            if pred.shape[1] != 0:
                submission_part = predict_multiclass_adaptive(pred, meta_te, action_thresholds)
                submission_list.append(submission_part)
            else:
                if verbose: print(f"  ERROR: no useful training data")
                
        except KeyError as e:
            if verbose: print(f'  ERROR: KeyError because of missing bodypart ({switch_tr}): {str(e)[:50]}')
            try:
                del data_te
            except:
                pass
            gc.collect()
        except Exception as e:
            if verbose: print(f'  ERROR: {str(e)[:80]}')
            try:
                del data_te
            except:
                pass
            gc.collect()


In [14]:
# %%time
f1_list = []
submission_list = []
for section in range(1, len(body_parts_tracked_list)): # skip index 0 (MABe22)
    body_parts_tracked_str = body_parts_tracked_list[section]
    try:
        body_parts_tracked = json.loads(body_parts_tracked_str)
        print(f"{section}. Processing videos with {body_parts_tracked}")
        if len(body_parts_tracked) > 5:
            body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
    
        # We read all training data which match the body parts tracked
        train_subset = train[train.body_parts_tracked == body_parts_tracked_str]
        single_mouse_list = []
        single_mouse_label_list = []
        single_mouse_meta_list = []
        mouse_pair_list = []
        mouse_pair_label_list = []
        mouse_pair_meta_list = []
    
        for switch, data, meta, label in generate_mouse_data(train_subset, 'train'):
            if switch == 'single':
                single_mouse_list.append(data)
                single_mouse_meta_list.append(meta)
                single_mouse_label_list.append(label)
            else:
                mouse_pair_list.append(data)
                mouse_pair_meta_list.append(meta)
                mouse_pair_label_list.append(label)
    
        # =============================================================================
        # OPTIMIZED ENSEMBLE: 5 STRONG MODELS (Based on Top Kaggle Notebooks)
        # Key improvements:
        # - Reduced from 7-9 to 5 diverse, powerful models
        # - Increased n_estimators (350-400 vs 200-300)
        # - Better regularization (min_child_samples: 20-40 vs 5)
        # - Dynamic sample sizes (300K-500K) for efficiency
        # - Stronger learning rates for faster convergence
        # =============================================================================
        
        base_n_samples = 600_000  # Base sample size
        models = []
        
        # Configure device for each model type
        lgbm_device = 'cpu'  # CPU for stability with imbalanced data
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"  🚀 OPTIMIZED ENSEMBLE: 5 STRONG MODELS")
            print(f"  LightGBM: CPU (stable for rare classes)")
            print(f"  XGBoost:  {'GPU' if GPU_AVAILABLE else 'CPU'}")
            print(f"  CatBoost: {'GPU' if CATBOOST_AVAILABLE and GPU_AVAILABLE else 'CPU'}")
            print(f"{'='*60}\n")
        
        # Base LightGBM parameters - STRONGER regularization
        lgbm_base_params = {
            'device': lgbm_device,
            'verbose': -1,
            'min_gain_to_split': 0.0,
            'min_sum_hessian_in_leaf': 1e-3,
        }
        
        # =============================================================================
        # MODEL 1: LightGBM - Strong Baseline (400 estimators)
        # =============================================================================
        models.append(make_pipeline(
            SimpleImputer(),
            StratifiedSubsetClassifier(
                lightgbm.LGBMClassifier(
                    n_estimators=400,           # Increased from 300
                    learning_rate=0.07,         # Increased from 0.06
                    min_child_samples=40,       # Increased from 5 for stability
                    num_leaves=31,
                    max_depth=-1,               # No limit, controlled by leaves
                    subsample=0.8,
                    colsample_bytree=0.8,
                    reg_alpha=0.0,
                    reg_lambda=0.0,
                    **lgbm_base_params,
                    random_state=SEED,
                    bagging_seed=SEED,
                    feature_fraction_seed=SEED,
                    data_random_seed=SEED
                ), int(base_n_samples / 1.3)    # ~460K samples
            )
        ))
        
        # =============================================================================
        # MODEL 2: LightGBM - Deeper Trees (300 estimators)
        # =============================================================================
        models.append(make_pipeline(
            SimpleImputer(),
            StratifiedSubsetClassifier(
                lightgbm.LGBMClassifier(
                    n_estimators=300,           # Good balance
                    learning_rate=0.1,          # Higher for faster convergence
                    min_child_samples=20,       # Increased from 5
                    num_leaves=63,              # More leaves for complexity
                    max_depth=8,                # Deeper trees
                    subsample=0.7,
                    colsample_bytree=0.9,
                    reg_alpha=0.1,              # Stronger regularization
                    reg_lambda=0.1,
                    **lgbm_base_params,
                    random_state=SEED+1,
                    bagging_seed=SEED+1,
                    feature_fraction_seed=SEED+1,
                    data_random_seed=SEED+1
                ), int(base_n_samples / 2)      # ~300K samples
            )
        ))
        
        # =============================================================================
        # MODEL 3: XGBoost - Strong GPU Accelerated (400 estimators)
        # =============================================================================
        xgb_params = {
            'n_estimators': 400,                # Increased from 250
            'learning_rate': 0.08,
            'max_depth': 6,
            'min_child_weight': 5,              # Increased from 1 for stability
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.0,
            'reg_lambda': 0.0,
            'tree_method': 'hist',
            'device': 'cuda:0' if GPU_AVAILABLE else 'cpu',
            'random_state': SEED,
            'n_jobs': -1,
            'eval_metric': 'logloss',
            'use_label_encoder': False
        }
        
        models.append(make_pipeline(
            SimpleImputer(),
            StratifiedSubsetClassifier(
                XGBClassifier(**xgb_params), int(base_n_samples / 1.5)  # ~400K samples
            )
        ))
        
        # =============================================================================
        # MODEL 4: CatBoost - Configuration 1 (400 iterations)
        # =============================================================================
        if CATBOOST_AVAILABLE:
            catboost_params_1 = {
                'iterations': 400,              # Increased from 180
                'learning_rate': 0.1,           # Increased from 0.08
                'depth': 6,
                'verbose': False,
                'allow_writing_files': False,
                'random_seed': SEED,
            }
            if GPU_AVAILABLE:
                catboost_params_1['task_type'] = 'GPU'
                catboost_params_1['devices'] = '0'
            
            models.append(make_pipeline(
                SimpleImputer(),
                StratifiedSubsetClassifier(
                    CatBoostClassifier(**catboost_params_1), base_n_samples  # 600K samples
                )
            ))
        
        # =============================================================================
        # MODEL 5: CatBoost - Configuration 2 (300 iterations)
        # =============================================================================
        if CATBOOST_AVAILABLE:
            catboost_params_2 = {
                'iterations': 300,
                'learning_rate': 0.1,
                'depth': 6,
                'verbose': False,
                'allow_writing_files': False,
                'random_seed': SEED+1,
            }
            if GPU_AVAILABLE:
                catboost_params_2['task_type'] = 'GPU'
                catboost_params_2['devices'] = '0'
            
            models.append(make_pipeline(
                SimpleImputer(),
                StratifiedSubsetClassifier(
                    CatBoostClassifier(**catboost_params_2), base_n_samples  # 600K samples
                )
            ))
        
        # =============================================================================
        # OPTIMIZED MODEL WEIGHTS
        # Based on analysis: LightGBM and XGBoost perform best, CatBoost adds diversity
        # =============================================================================
        if CATBOOST_AVAILABLE:
            # 5 models: 2 LGBM + 1 XGB + 2 CatBoost
            model_weights = [0.25, 0.20, 0.30, 0.15, 0.10]
        else:
            # 3 models: 2 LGBM + 1 XGB
            model_weights = [0.35, 0.30, 0.35]
        
        if verbose:
            print(f"  ✓ Ensemble configured: {len(models)} models")
            print(f"    Weights: {model_weights}")
            print(f"    Sample sizes: {[f'{int(base_n_samples/1.3):,}', f'{int(base_n_samples/2):,}', f'{int(base_n_samples/1.5):,}', f'{base_n_samples:,}', f'{base_n_samples:,}'][:len(models)]}")
        
        # ---------------------------
        # ---------------------------
        # Predict single-mouse actions
        if len(single_mouse_list) > 0:
            # Concatenate all batches
            single_mouse = pd.concat(single_mouse_list)
            single_mouse_label = pd.concat(single_mouse_label_list)
            single_mouse_meta = pd.concat(single_mouse_meta_list)
            del single_mouse_list, single_mouse_label_list, single_mouse_meta_list
            assert len(single_mouse) == len(single_mouse_label)
            assert len(single_mouse) == len(single_mouse_meta)
            
            # Get FPS for feature transformation
            fps = single_mouse_meta['fps'].iloc[0] if 'fps' in single_mouse_meta.columns else 30.0
            
            # Transform with FPS awareness
            X_tr = transform_single(single_mouse, body_parts_tracked, fps)
            del single_mouse
            print(f"{X_tr.shape=}")
    
            if validate_or_submit == 'validate':
                cross_validate_classifier(models, X_tr, single_mouse_label, single_mouse_meta)
            else:
                submit(body_parts_tracked_str, 'single', models, model_weights, X_tr, single_mouse_label, single_mouse_meta)
            del X_tr
            gc.collect()
                
        # Predict mouse-pair actions
        if len(mouse_pair_list) > 0:
            # Concatenate all batches
            mouse_pair = pd.concat(mouse_pair_list)
            mouse_pair_label = pd.concat(mouse_pair_label_list)
            mouse_pair_meta = pd.concat(mouse_pair_meta_list)
            del mouse_pair_list, mouse_pair_label_list, mouse_pair_meta_list
            assert len(mouse_pair) == len(mouse_pair_label)
            assert len(mouse_pair) == len(mouse_pair_meta)
        
            # Get FPS for feature transformation
            fps = mouse_pair_meta['fps'].iloc[0] if 'fps' in mouse_pair_meta.columns else 30.0
            
            # Transform with FPS awareness
            X_tr = transform_pair(mouse_pair, body_parts_tracked, fps)
            del mouse_pair
            print(f"{X_tr.shape=}")
    
            if validate_or_submit == 'validate':
                cross_validate_classifier(models, X_tr, mouse_pair_label, mouse_pair_meta)
            else:
                submit(body_parts_tracked_str, 'pair', models, model_weights, X_tr, mouse_pair_label, mouse_pair_meta)
            del X_tr
            gc.collect()
                
    except Exception as e:
        print(f'***Exception*** {e}')
    print()


1. Processing videos with ['body_center', 'ear_left', 'ear_right', 'headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 'lateral_left', 'lateral_right', 'neck', 'nose', 'tail_base', 'tail_midpoint', 'tail_tip']

  🚀 OPTIMIZED ENSEMBLE: 5 STRONG MODELS
  LightGBM: CPU (stable for rare classes)
  XGBoost:  GPU
  CatBoost: GPU

  ✓ Ensemble configured: 5 models
    Weights: [0.25, 0.2, 0.3, 0.15, 0.1]
    Sample sizes: ['461,538', '300,000', '400,000', '600,000', '600,000']
X_tr.shape=(544859, 115)
n_videos: 1, n_models: 5
video with missing values 438887472 test 529471 frames
- test single 438887472 1
  actions found: 15
- test single 438887472 2
  actions found: 89
- test single 438887472 3
  actions found: 58
- test single 438887472 4
  actions found: 131
X_tr.shape=(1744248, 142)
n_videos: 1, n_models: 5
video with missi

In [15]:
def robustify(submission, dataset, traintest, traintest_directory=None):
    """Ensure that the submission conforms to the three rules"""
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"

    # Rule 1: Ensure that start_frame >= stop_frame
    old_submission = submission.copy()
    submission = submission[submission.start_frame < submission.stop_frame]
    if len(submission) != len(old_submission):
        print("ERROR: Dropped frames with start >= stop")
    
    # Rule 2: Avoid multiple predictions for the same frame from one agent/target pair
    old_submission = submission.copy()
    group_list = []
    for _, group in submission.groupby(['video_id', 'agent_id', 'target_id']):
        group = group.sort_values('start_frame')
        mask = np.ones(len(group), dtype=bool)
        last_stop_frame = 0
        for i, (_, row) in enumerate(group.iterrows()):
            if row['start_frame'] < last_stop_frame:
                mask[i] = False
            else:
                last_stop_frame = row['stop_frame']
        group_list.append(group[mask])
    submission = pd.concat(group_list)
    if len(submission) != len(old_submission):
        print("ERROR: Dropped duplicate frames")

    # Rule 3: Submit something for every video
    # Fill missing videos as in https://www.kaggle.com/code/ambrosm/mabe-validated-baseline-without-machine-learning
    s_list = []
    for idx, row in dataset.iterrows():
        lab_id = row['lab_id']
        if lab_id.startswith('MABe22'):
            continue
        video_id = row['video_id']
        if (submission.video_id == video_id).any():
            continue

        if verbose: print(f"Video {video_id} has no predictions.")
        
        # Load video
        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
    
        # Determine the behaviors of this video
        # Determine the behaviors of this video
        val = row['behaviors_labeled']

        # 1. Nếu NaN -> không có label hành vi -> bỏ qua rule 3 cho video này
        if pd.isna(val):
            if verbose:
                print(f"WARN: behaviors_labeled is NaN for video {video_id}, skip filling.")
            continue

        # 2. Nếu là string -> parse bằng literal_eval (an toàn hơn eval)
        if isinstance(val, str):
            try:
                vid_behaviors = ast.literal_eval(val)
            except Exception as e:
                print(f"WARN: could not parse behaviors_labeled for video {video_id}: {e}")
                continue

        # 3. Nếu đã là list/tuple -> dùng luôn
        elif isinstance(val, (list, tuple, np.ndarray)):
            vid_behaviors = val

        # 4. Kiểu khác -> bỏ qua
        else:
            print(f"WARN: unexpected type for behaviors_labeled in video {video_id}: {type(val)}")
            continue

        # Phần còn lại giữ nguyên
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
    
        # Determine start_frame and stop_frame
        start_frame = vid.video_frame.min()
        stop_frame = vid.video_frame.max() + 1
    
        # Predict all possible actions as often as possible
        for (agent, target), actions in vid_behaviors.groupby(['agent', 'target']):
            batch_length = int(np.ceil((stop_frame - start_frame) / len(actions)))
            for i, (_, action_row) in enumerate(actions.iterrows()):
                batch_start = start_frame + i * batch_length
                batch_stop = min(batch_start + batch_length, stop_frame)
                s_list.append((video_id, agent, target, action_row['action'], batch_start, batch_stop))

    if len(s_list) > 0:
        submission = pd.concat([
            submission,
            pd.DataFrame(s_list, columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        ])
        print("ERROR: Filled empty videos")

    submission = submission.reset_index(drop=True)
    return submission


In [16]:
if validate_or_submit == 'validate':
    # Score the oof predictions with the competition scoring function
    submission = pd.concat(submission_list)
    submission_robust = robustify(submission, train, 'train')
    print(f"# OOF score with competition metric: {score(solution, submission_robust, ''):.4f}")

    f1_df = pd.DataFrame(f1_list, columns=['body_parts_tracked_str', 'action', 'binary F1 score'])
    print(f"# Average of {len(f1_df)} binary F1 scores {f1_df['binary F1 score'].mean():.4f}")
    # with pd.option_context('display.max_rows', 500):
    #     display(f1_df)

In [17]:
if validate_or_submit != 'validate':
    if len(submission_list) > 0:
        submission = pd.concat(submission_list)
    else:
        submission = pd.DataFrame(
            dict(
                video_id=438887472,
                agent_id='mouse1',
                target_id='self',
                action='rear',
                start_frame=278,
                stop_frame=500
            ), index=[44])
    if validate_or_submit == 'submit':
        submission_robust = robustify(submission, test, 'test')
    else:
        submission_robust = robustify(submission, stresstest, 'stresstest', 'stresstest_tracking')
    submission_robust.index.name = 'row_id'
    submission_robust.to_csv('submission.csv')
    !head submission.csv

row_id,video_id,agent_id,target_id,action,start_frame,stop_frame
0,438887472,mouse1,mouse3,approach,2286,2291
1,438887472,mouse1,mouse4,attack,1417,1419
2,438887472,mouse1,mouse4,attack,1425,1442
3,438887472,mouse1,mouse4,attack,1452,1486
4,438887472,mouse1,mouse4,attack,1495,1503
5,438887472,mouse1,mouse4,attack,1505,1524
6,438887472,mouse1,mouse4,attack,1534,1537
7,438887472,mouse1,mouse4,submit,2381,2389
8,438887472,mouse1,mouse4,avoid,2564,2581
